In [20]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import string 

In [21]:
def create_map(ccch_path, charities_path, maos_path, dts_path, coordinates, zoom_start, la_name, businesses_path, iz_info_save_location, iz_table_dash_save_location, map_save_location):
    # Reads in all data
    pc_lookup = pd.read_csv('../data/scotland/postcode_lookup.csv', low_memory=False)
    pc_lookup.drop_duplicates(inplace=True)
    ccch = pd.read_csv(ccch_path)
    charities = pd.read_csv(charities_path)
    businesses = pd.read_csv(businesses_path)
    maos = pd.read_csv(maos_path)
    dts = pd.read_csv(dts_path)
    simd = pd.read_csv("../data/scotland/SIMD_2020.csv")
    le = pd.read_csv("../data/scotland/scotpho_data_extract.csv")
    dz_lookup = pd.read_csv("../data/scotland/2011_dz_lookup.csv")
    scotland_iz = gpd.read_file("../data/scotland/2011_IZ_Boundary/SG_IntermediateZone_Bdry_2011.shp")
    pop_est = pd.read_csv("../data/scotland/dz_2020_pop_est.csv")

    # Gets pop est by iz instead of dz
    pop_est = pop_est.merge(dz_lookup, left_on="DataZone2011Code", right_on="DZ2011_Code")
    pop_est["Total population"] = pop_est["Total population"].str.replace(",", "").astype(int)
    pop_est = pop_est.groupby("IZ2011_Code")["Total population"].sum().reset_index()
    
    # Gets list of codes for each local authority
    la_dz = dz_lookup.loc[dz_lookup["LA_Name"] == la_name].copy()
    la_iz_codes = la_dz["IZ2011_Code"].to_list()
    la_iz_layer = scotland_iz[scotland_iz["InterZone"].isin(la_iz_codes)]
    la_dz_codes = la_dz["DZ2011_Code"].to_list()

    # Gets latitude and longitude for community spaces from postcode
    ccch = ccch.rename(columns={"name": "Name", "postcode": "Postcode"})
    ccch = ccch.loc[:, ["Name", "Postcode"]].copy()
    ccch = pd.merge(ccch, pc_lookup, left_on="Postcode", right_on="Postcode")
    ccch = ccch.loc[:, ["Name", "Postcode", "IntermediateZone2011Code", "Latitude", "Longitude"]].copy()

    # Formats business df to separate iz code and name to two columns
    businesses.rename(columns={"2011 scottish intermediate zone": "IZ2011_Code"}, inplace=True)
    businesses["IZ_Code"] = ""
    businesses["IZ_Name"] = ""
    new_cols = businesses.IZ2011_Code.str.split(":", expand=True)
    businesses["IZ_Code"] = new_cols[0].apply(lambda x: x.rstrip())
    businesses["IZ_Name"] = new_cols[1].apply(lambda x: x.rstrip())
    businesses.drop(columns=["IZ2011_Code"], inplace=True)
    businesses["IZ_Code"].apply(lambda x: str(x))
    businesses = businesses.loc[:, ["IZ_Code", "Total"]].copy()

    # Takes mean dz simd for each iz
    simd = simd.loc[simd["Units"] == "Decile"].copy()
    simd_la = simd.loc[simd["FeatureCode"].isin(la_dz_codes)]
    simd_simd_la = simd_la.loc[simd["SIMD Domain"] == "SIMD"].copy()
    dz_min = dz_lookup.loc[:, ["DZ2011_Code", "IZ2011_Code"]].copy()
    simd_iz = pd.merge(simd_simd_la, dz_min, left_on="FeatureCode", right_on="DZ2011_Code")
    simd_iz = simd_iz.groupby("IZ2011_Code")["Value"].mean().reset_index()
    simd_iz["Value"] = simd_iz["Value"].round().astype(int)
    
    # Gets num of charities for each iz
    charities_by_iz = charities.groupby("IntermediateZone2011Code")["Charity Number"].count().reset_index()
    charities_by_iz.rename(columns={"Charity Number": "Number of charities"}, inplace=True)

    # Gets female and male life expectancy for each iz
    female_le = le.loc[(le["indicator"] == "Life expectancy, females") & (le["area_type"] == "Intermediate zone") & (le["period"] == "2016 to 2020 (5 year aggregate)")].copy()
    female_le = female_le.loc[:, ["area_code", "measure"]].copy()
    female_le.rename(columns={"measure": "Female life expectancy"}, inplace=True)
    female_le["Female life expectancy"] = female_le["Female life expectancy"].apply(lambda x: round(x))
    male_le = le.loc[(le["indicator"] == "Life expectancy, males") & (le["area_type"] == "Intermediate zone") & (le["period"] == "2016 to 2020 (5 year aggregate)")].copy()
    male_le = male_le.loc[:, ["area_code", "measure"]].copy()
    male_le.rename(columns={"measure": "Male life expectancy"}, inplace=True)
    male_le["Male life expectancy"] = male_le["Male life expectancy"].apply(lambda x: round(x))

    # Merges all of above for the tooltips and table
    simd_iz_geom = simd_iz.merge(la_iz_layer, left_on="IZ2011_Code", right_on="InterZone")
    iz_info = pd.merge(simd_iz_geom, businesses, left_on="IZ2011_Code", right_on="IZ_Code")
    iz_info = pd.merge(iz_info, pop_est, left_on="IZ2011_Code", right_on="IZ2011_Code")
    iz_info = iz_info.loc[:, ["IZ2011_Code", "Name", "Value", "Total", "Total population", "geometry"]].copy()
    iz_info = iz_info.merge(charities_by_iz, left_on="IZ2011_Code", right_on="IntermediateZone2011Code", how="left")
    iz_info = iz_info.merge(female_le, left_on="IZ2011_Code", right_on="area_code", how="left")
    iz_info = iz_info.merge(male_le, left_on="IZ2011_Code", right_on="area_code", how="left")
    iz_info.drop(columns=["area_code_x", "area_code_y"], inplace=True)
    iz_info.rename(columns={"Value": "Mean SIMD Decile", "Total": "Number of businesses"}, inplace=True)
    
    # Imputes missing values
    iz_info["Number of charities"] = iz_info["Number of charities"].fillna(0)
    imp_m_le = round(iz_info["Male life expectancy"].mean(), 0)
    imp_f_le = round(iz_info["Female life expectancy"].mean(), 0)
    iz_info["Female life expectancy"] = iz_info["Female life expectancy"].fillna(imp_f_le)
    iz_info["Male life expectancy"] = iz_info["Male life expectancy"].fillna(imp_m_le)

    # Calculates businesses and charities per 1000 
    iz_info["Local charities per 1000 people"] = ((iz_info["Number of charities"] / iz_info["Total population"]) * 1000).round(2)
    iz_info["Businesses per 1000 people"] = ((iz_info["Number of businesses"] / iz_info["Total population"]) * 1000).round(0)
   
    # Fixes column names
    iz_info.rename(columns={"Name": "Community"}, inplace=True)
    
    # Adds ccch_per_1000 column to iz_info
    ccch_iz = ccch.loc[:, ["Name", "IntermediateZone2011Code"]].copy()
    no_of_ccch = pd.merge(ccch_iz, iz_info, left_on="IntermediateZone2011Code", right_on="IZ2011_Code", how="right")
    no_of_ccch = no_of_ccch.groupby("IZ2011_Code").count().reset_index()
    no_of_ccch = no_of_ccch.loc[:, ["IZ2011_Code", "Name"]].copy()
    no_of_ccch.rename(columns={"Name": "Number of community spaces"}, inplace=True)
    iz_info = pd.merge(iz_info, no_of_ccch, left_on="IZ2011_Code", right_on="IZ2011_Code")
    iz_info["Community spaces per 1000 people"] = ((iz_info["Number of community spaces"] / iz_info["Total population"]) * 1000).round(2)
    iz_info.drop(columns=["IntermediateZone2011Code"], inplace=True)

    iz_info_for_dash_table = iz_info.loc[:, ["Community", "Mean SIMD Decile", "Female life expectancy", "Male life expectancy", "Businesses per 1000 people", "Local charities per 1000 people", "Community spaces per 1000 people"]].copy()
    iz_info_for_dash_table.to_csv(iz_table_dash_save_location, index=False)
    iz_info.to_csv(iz_info_save_location, index=False)

    # Creates initial map
    m = folium.Map(location=coordinates, zoom_start=zoom_start, tiles=None, attr="Imagery © <a href='https://www.mapbox.com/about/maps/'>Mapbox</a>, Map data © <a href='http://www.openstreetmap.org/copyright'>OpenStreetMap</a> contributors.")
    base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
    folium.TileLayer(tiles="openstreetmap").add_to(base_map)
    base_map.add_to(m)

    # Prepares layers to plot on map
    ccch = gpd.GeoDataFrame(ccch)
    charities = gpd.GeoDataFrame(charities)
    maos = gpd.GeoDataFrame(maos)
    dts = gpd.GeoDataFrame(dts)
    iz_info = gpd.GeoDataFrame(iz_info)

    # Adds point layers to map
    def create_marker_layer(gdf, marker_cluster, icon_color, icon_image):
        for _x, resource in gdf.iterrows():
            html = folium.Html(f"<p>{resource['Name'].translate(str.maketrans('', '', string.punctuation))}</p>", script=True)
            folium.Marker([resource.Latitude, resource.Longitude], popup=folium.Popup(html, parse_html=True), icon=folium.Icon(color=icon_color, icon_color="white", icon=icon_image, prefix="fa")).add_to(marker_cluster)

    # To make check boxes, set overlay to True
    ccch_markers = MarkerCluster(name="Community Spaces", data="community_centres_and_halls", overlay=False).add_to(m)
    charity_markers = MarkerCluster(name="Charities", data="charities_gdf", overlay=False).add_to(m)
    dt_markers = MarkerCluster(name="Development Trusts", data="dts_gdf", overlay=False).add_to(m)
    mao_markers = MarkerCluster(name="Mutual Aid Organisations", data="maos_gdf", overlay=False).add_to(m)
    create_marker_layer(ccch, ccch_markers, "green", "users")
    create_marker_layer(charities, charity_markers, "red", "heart")
    create_marker_layer(dts, dt_markers, "lightgray", "leaf")
    create_marker_layer(maos, mao_markers, "lightgreen", "sitemap")
    
    # Adds iz outlines to map
    la_iz_layer = la_iz_layer.to_crs(epsg=27700)
    style1 = {"fillColor": "#063A47", "color": "#000000", "weight": 0.7}
    folium.GeoJson(data=la_iz_layer["geometry"], style_function=lambda x: style1, name=la_name, control=False).add_to(m)

    # Adds tooltip layer to map
    style_function = lambda x: {"fillColor": '#063A47', 
                            'color':'#000000', 
                            'fillOpacity': 0.02, 
                            'weight': 0.01}
    highlight_function = lambda x: {'fillColor': '#063A47', 
                                        'color':'#063A47', 
                                        'fillOpacity': 0.5, 
                                        'weight': 0.1}                              

    iz_geojson = folium.features.GeoJson(
            iz_info,
            style_function=style_function,
            control=True,
            highlight_function=highlight_function, 
            name="Intermediate Zone Information",
            tooltip=folium.features.GeoJsonTooltip(
                fields=['Community', 'Mean SIMD Decile', 'Female life expectancy', 'Male life expectancy', 'Businesses per 1000 people', 'Local charities per 1000 people'],
                aliases=['Community: ', 'Mean SIMD Decile: ', 'Female life expectancy: ', 'Male life expectancy: ', 'Businesses per 1000 people: ', 'Locally operating charities per 1000 people: '],
                labels=True,
                sticky=True, 
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
            )
        )

    m.add_child(iz_geojson)
    m.keep_in_front(iz_geojson)
    folium.LayerControl(collapsed=False).add_to(m)
    m.save(map_save_location)

In [22]:
# Aberdeen City
create_map("../data/aberdeen_city/ccch_aberdeen_city.csv", "../data/aberdeen_city/charities_aberdeen_city.csv", "../data/aberdeen_city/maos_aberdeen_city.csv", "../data/aberdeen_city/dts_aberdeen_city.csv", (57.149651, -2.099075), 11, "Aberdeen City", "../data/aberdeen_city/businesses_aberdeen_city.csv", "../data/aberdeen_city/iz_info_aberdeen_city.csv", "../data/aberdeen_city/iz_info_dash_table_aberdeen_city.csv", "../assets/aberdeen_city_map.html")

In [23]:
# Aberdeenshire
create_map("../data/aberdeenshire/ccch_aberdeenshire.csv", "../data/aberdeenshire/charities_aberdeenshire.csv", "../data/aberdeenshire/maos_aberdeenshire.csv", "../data/aberdeenshire/dts_aberdeenshire.csv", (57.28446, -2.37736), 8, "Aberdeenshire", "../data/aberdeenshire/businesses_aberdeenshire.csv", "../data/aberdeenshire/iz_info_aberdeenshire.csv", "../data/aberdeenshire/iz_info_dash_table_aberdeenshire.csv", "../assets/aberdeenshire_map.html")

In [24]:
# Angus 
create_map("../data/angus/ccch_angus.csv", "../data/angus/charities_angus.csv", "../data/angus/maos_angus.csv", "../data/angus/dts_angus.csv", (56.666664, -2.916663), 9, "Angus", "../data/angus/businesses_angus.csv", "../data/angus/iz_info_angus.csv", "../data/angus/iz_info_dash_table_angus.csv", "../assets/angus_map.html")

In [25]:
# Argyll and Bute
create_map("../data/argyll_and_bute/ccch_argyll_and_bute.csv", "../data/argyll_and_bute/charities_argyll_and_bute.csv", "../data/argyll_and_bute/maos_argyll_and_bute.csv", "../data/argyll_and_bute/dts_argyll_and_bute.csv", (56.03693, -5.43679), 8, "Argyll and Bute", "../data/argyll_and_bute/businesses_argyll_and_bute.csv", "../data/argyll_and_bute/iz_info_argyll_and_bute.csv", "../data/argyll_and_bute/iz_info_dash_table_argyll_and_bute.csv", "../assets/argyll_and_bute_map.html")

In [26]:
# City of Edinburgh
create_map("../data/city_of_edinburgh/ccch_city_of_edinburgh.csv", "../data/city_of_edinburgh/charities_city_of_edinburgh.csv", "../data/city_of_edinburgh/maos_city_of_edinburgh.csv", "../data/city_of_edinburgh/dts_city_of_edinburgh.csv", (55.953251, -3.188267), 10, "City of Edinburgh", "../data/city_of_edinburgh/businesses_city_of_edinburgh.csv", "../data/city_of_edinburgh/iz_info_city_of_edinburgh.csv", "../data/city_of_edinburgh/iz_info_dash_table_city_of_edinburgh.csv", "../assets/city_of_edinburgh_map.html")

In [27]:
# Clackmannanshire
create_map("../data/clackmannanshire/ccch_clackmannanshire.csv", "../data/clackmannanshire/charities_clackmannanshire.csv", "../data/clackmannanshire/maos_clackmannanshire.csv", "../data/clackmannanshire/dts_clackmannanshire.csv", (56.166666, -3.749997), 11, "Clackmannanshire", "../data/clackmannanshire/businesses_clackmannanshire.csv", "../data/clackmannanshire/iz_info_clackmannanshire.csv", "../data/clackmannanshire/iz_info_dash_table_clackmannanshire.csv", "../assets/clackmannanshire_map.html")

In [28]:
# Dumfries and Galloway
create_map("../data/dumfries_and_galloway/ccch_dumfries_and_galloway.csv", "../data/dumfries_and_galloway/charities_dumfries_and_galloway.csv", "../data/dumfries_and_galloway/maos_dumfries_and_galloway.csv", "../data/dumfries_and_galloway/dts_dumfries_and_galloway.csv", (54.988285, -3.857784), 9, "Dumfries and Galloway", "../data/dumfries_and_galloway/businesses_dumfries_and_galloway.csv", "../data/dumfries_and_galloway/iz_info_dumfries_and_galloway.csv", "../data/dumfries_and_galloway/iz_info_dash_table_dumfries_and_galloway.csv", "../assets/dumfries_and_galloway_map.html")

In [29]:
# Dundee City
create_map("../data/dundee_city/ccch_dundee_city.csv", "../data/dundee_city/charities_dundee_city.csv", "../data/dundee_city/maos_dundee_city.csv", "../data/dundee_city/dts_dundee_city.csv", (56.46913, -2.97489), 12, "Dundee City", "../data/dundee_city/businesses_dundee_city.csv", "../data/dundee_city/iz_info_dundee_city.csv", "../data/dundee_city/iz_info_dash_table_dundee_city.csv", "../assets/dundee_city_map.html")

In [30]:
# East Ayrshire
create_map("../data/east_ayrshire/ccch_east_ayrshire.csv", "../data/east_ayrshire/charities_east_ayrshire.csv", "../data/east_ayrshire/maos_east_ayrshire.csv", "../data/east_ayrshire/dts_east_ayrshire.csv", (55.5, -4.25), 9, "East Ayrshire", "../data/east_ayrshire/businesses_east_ayrshire.csv", "../data/east_ayrshire/iz_info_east_ayrshire.csv", "../data/east_ayrshire/iz_info_dash_table_east_ayrshire.csv", "../assets/east_ayrshire_map.html")

In [31]:
# East Dunbartonshire
create_map("../data/east_dunbartonshire/ccch_east_dunbartonshire.csv", "../data/east_dunbartonshire/charities_east_dunbartonshire.csv", "../data/east_dunbartonshire/maos_east_dunbartonshire.csv", "../data/east_dunbartonshire/dts_east_dunbartonshire.csv", (55.93333, -4.2), 11, "East Dunbartonshire", "../data/east_dunbartonshire/businesses_east_dunbartonshire.csv", "../data/east_dunbartonshire/iz_info_east_dunbartonshire.csv", "../data/east_dunbartonshire/iz_info_dash_table_east_dunbartonshire.csv", "../assets/east_dunbartonshire_map.html")

In [32]:
# East Lothian
create_map("../data/east_lothian/ccch_east_lothian.csv", "../data/east_lothian/charities_east_lothian.csv", "../data/east_lothian/maos_east_lothian.csv", "../data/east_lothian/dts_east_lothian.csv", (55.955621, -2.778815), 9, "East Lothian", "../data/east_lothian/businesses_east_lothian.csv", "../data/east_lothian/iz_info_east_lothian.csv", "../data/east_lothian/iz_info_dash_table_east_lothian.csv", "../assets/east_lothian_map.html")

In [33]:
# East Renfrewshire
create_map("../data/east_renfrewshire/ccch_east_renfrewshire.csv", "../data/east_renfrewshire/charities_east_renfrewshire.csv", "../data/east_renfrewshire/maos_east_renfrewshire.csv", "../data/east_renfrewshire/dts_east_renfrewshire.csv", (55.745, -4.413), 11, "East Renfrewshire", "../data/east_renfrewshire/businesses_east_renfrewshire.csv", "../data/east_renfrewshire/iz_info_east_renfrewshire.csv", "../data/east_renfrewshire/iz_info_dash_table_east_renfrewshire.csv", "../assets/east_renfrewshire_map.html")

In [34]:
# Falkirk
create_map("../data/falkirk/ccch_falkirk.csv", "../data/falkirk/charities_falkirk.csv", "../data/falkirk/maos_falkirk.csv", "../data/falkirk/dts_falkirk.csv", (56.0021, -3.78535), 11, "Falkirk", "../data/falkirk/businesses_falkirk.csv", "../data/falkirk/iz_info_falkirk.csv", "../data/falkirk/iz_info_dash_table_falkirk.csv", "../assets/falkirk_map.html")

In [35]:
# Fife
create_map("../data/fife/ccch_fife.csv", "../data/fife/charities_fife.csv", "../data/fife/maos_fife.csv", "../data/fife/dts_fife.csv", (56.2082, -3.1495), 10, "Fife", "../data/fife/businesses_fife.csv", "../data/fife/iz_info_fife.csv", "../data/fife/iz_info_dash_table_fife.csv", "../assets/fife_map.html")

In [36]:
# Glasgow City
create_map("../data/glasgow_city/ccch_glasgow_city.csv", "../data/glasgow_city/charities_glasgow_city.csv", "../data/glasgow_city/maos_glasgow_city.csv", "../data/glasgow_city/dts_glasgow_city.csv", (55.86515, -4.25763), 11, "Glasgow City", "../data/glasgow_city/businesses_glasgow_city.csv", "../data/glasgow_city/iz_info_glasgow_city.csv", "../data/glasgow_city/iz_info_dash_table_glasgow_city.csv", "../assets/glasgow_city_map.html")

In [37]:
# Highland
create_map("../data/highland/ccch_highland.csv", "../data/highland/charities_highland.csv", "../data/highland/maos_highland.csv", "../data/highland/dts_highland.csv", (57.499998, -5.0), 7, "Highland", "../data/highland/businesses_highland.csv", "../data/highland/iz_info_highland.csv", "../data/highland/iz_info_dash_table_highland.csv", "../assets/highland_map.html")

In [38]:
# Inverclyde
create_map("../data/inverclyde/ccch_inverclyde.csv", "../data/inverclyde/charities_inverclyde.csv", "../data/inverclyde/maos_inverclyde.csv", "../data/inverclyde/dts_inverclyde.csv", (55.8999964, -4.749997), 11, "Inverclyde", "../data/inverclyde/businesses_inverclyde.csv", "../data/inverclyde/iz_info_inverclyde.csv", "../data/inverclyde/iz_info_dash_table_inverclyde.csv", "../assets/inverclyde_map.html")

In [39]:
# Midlothian
create_map("../data/midlothian/ccch_midlothian.csv", "../data/midlothian/charities_midlothian.csv", "../data/midlothian/maos_midlothian.csv", "../data/midlothian/dts_midlothian.csv", (55.8292247, -3.1338428), 11, "Midlothian", "../data/midlothian/businesses_midlothian.csv", "../data/midlothian/iz_info_midlothian.csv", "../data/midlothian/iz_info_dash_table_midlothian.csv", "../assets/midlothian_map.html")

In [40]:
# Moray
create_map("../data/moray/ccch_moray.csv", "../data/moray/charities_moray.csv", "../data/moray/maos_moray.csv", "../data/moray/dts_moray.csv", (57.416665, -3.249999), 9, "Moray", "../data/moray/businesses_moray.csv", "../data/moray/iz_info_moray.csv", "../data/moray/iz_info_dash_table_moray.csv", "../assets/moray_map.html")

In [41]:
# Na h-Eileanan Siar
create_map("../data/na_h-eileanan_siar/ccch_na_h-eileanan_siar.csv", "../data/na_h-eileanan_siar/charities_na_h-eileanan_siar.csv", "../data/na_h-eileanan_siar/maos_na_h-eileanan_siar.csv", "../data/na_h-eileanan_siar/dts_na_h-eileanan_siar.csv", (57.76, -7.02), 8, "Na h-Eileanan Siar", "../data/na_h-eileanan_siar/businesses_na_h-eileanan_siar.csv", "../data/na_h-eileanan_siar/iz_info_na_h-eileanan_siar.csv", "../data/na_h-eileanan_siar/iz_info_dash_table_na_h-eileanan_siar.csv", "../assets/na_h-eileanan_siar_map.html")

In [42]:
# North Ayrshire
create_map("../data/north_ayrshire/ccch_north_ayrshire.csv", "../data/north_ayrshire/charities_north_ayrshire.csv", "../data/north_ayrshire/maos_north_ayrshire.csv", "../data/north_ayrshire/dts_north_ayrshire.csv", (55.6416731, -4.75946), 10, "North Ayrshire", "../data/north_ayrshire/businesses_north_ayrshire.csv", "../data/north_ayrshire/iz_info_north_ayrshire.csv", "../data/north_ayrshire/iz_info_dash_table_north_ayrshire.csv", "../assets/north_ayrshire_map.html")

In [43]:
# North Lanarkshire
create_map("../data/north_lanarkshire/ccch_north_lanarkshire.csv", "../data/north_lanarkshire/charities_north_lanarkshire.csv", "../data/north_lanarkshire/maos_north_lanarkshire.csv", "../data/north_lanarkshire/dts_north_lanarkshire.csv", (55.829, -3.922), 11, "North Lanarkshire", "../data/north_lanarkshire/businesses_north_lanarkshire.csv", "../data/north_lanarkshire/iz_info_north_lanarkshire.csv", "../data/north_lanarkshire/iz_info_dash_table_north_lanarkshire.csv", "../assets/north_lanarkshire_map.html")

In [44]:
# Orkney Islands
create_map("../data/orkney_islands/ccch_orkney_islands.csv", "../data/orkney_islands/charities_orkney_islands.csv", "../data/orkney_islands/maos_orkney_islands.csv", "../data/orkney_islands/dts_orkney_islands.csv", (59.0, -3.0), 9, "Orkney Islands", "../data/orkney_islands/businesses_orkney_islands.csv", "../data/orkney_islands/iz_info_orkney_islands.csv", "../data/orkney_islands/iz_info_dash_table_orkney_islands.csv", "../assets/orkney_islands_map.html")

In [45]:
# Perth and Kinross
create_map("../data/perth_and_kinross/ccch_perth_and_kinross.csv", "../data/perth_and_kinross/charities_perth_and_kinross.csv", "../data/perth_and_kinross/maos_perth_and_kinross.csv", "../data/perth_and_kinross/dts_perth_and_kinross.csv", (56.4744, -3.5284), 9, "Perth and Kinross", "../data/perth_and_kinross/businesses_perth_and_kinross.csv", "../data/perth_and_kinross/iz_info_perth_and_kinross.csv", "../data/perth_and_kinross/iz_info_dash_table_perth_and_kinross.csv", "../assets/perth_and_kinross_map.html")

In [46]:
# Renfrewshire
create_map("../data/renfrewshire/ccch_renfrewshire.csv", "../data/renfrewshire/charities_renfrewshire.csv", "../data/renfrewshire/maos_renfrewshire.csv", "../data/renfrewshire/dts_renfrewshire.csv", (55.829858, -4.542838), 10, "Renfrewshire", "../data/renfrewshire/businesses_renfrewshire.csv", "../data/renfrewshire/iz_info_renfrewshire.csv", "../data/renfrewshire/iz_info_dash_table_renfrewshire.csv", "../assets/renfrewshire_map.html")

In [47]:
# Scottish Borders
create_map("../data/scottish_borders/ccch_scottish_borders.csv", "../data/scottish_borders/charities_scottish_borders.csv", "../data/scottish_borders/maos_scottish_borders.csv", "../data/scottish_borders/dts_scottish_borders.csv", (55.5486, -2.7861), 9, "Scottish Borders", "../data/scottish_borders/businesses_scottish_borders.csv", "../data/scottish_borders/iz_info_scottish_borders.csv", "../data/scottish_borders/iz_info_dash_table_scottish_borders.csv", "../assets/scottish_borders_map.html")

In [48]:
# Shetland Islands
create_map("../data/shetland_islands/ccch_shetland_islands.csv", "../data/shetland_islands/charities_shetland_islands.csv", "../data/shetland_islands/maos_shetland_islands.csv", "../data/shetland_islands/dts_shetland_islands.csv", (60.1546, -1.1478), 8, "Shetland Islands", "../data/shetland_islands/businesses_shetland_islands.csv", "../data/shetland_islands/iz_info_shetland_islands.csv", "../data/shetland_islands/iz_info_dash_table_shetland_islands.csv", "../assets/shetland_islands_map.html")

In [49]:
# South Ayrshire
create_map("../data/south_ayrshire/ccch_south_ayrshire.csv", "../data/south_ayrshire/charities_south_ayrshire.csv", "../data/south_ayrshire/maos_south_ayrshire.csv", "../data/south_ayrshire/dts_south_ayrshire.csv", (55.1589, -4.6292), 9, "South Ayrshire", "../data/south_ayrshire/businesses_south_ayrshire.csv", "../data/south_ayrshire/iz_info_south_ayrshire.csv", "../data/south_ayrshire/iz_info_dash_table_south_ayrshire.csv", "../assets/south_ayrshire_map.html")

In [50]:
# South Lanarkshire
create_map("../data/south_lanarkshire/ccch_south_lanarkshire.csv", "../data/south_lanarkshire/charities_south_lanarkshire.csv", "../data/south_lanarkshire/maos_south_lanarkshire.csv", "../data/south_lanarkshire/dts_south_lanarkshire.csv", (55.6736, -3.7820), 9, "South Lanarkshire", "../data/south_lanarkshire/businesses_south_lanarkshire.csv", "../data/south_lanarkshire/iz_info_south_lanarkshire.csv", "../data/south_lanarkshire/iz_info_dash_table_south_lanarkshire.csv", "../assets/south_lanarkshire_map.html")

In [51]:
# Stirling
create_map("../data/stirling/ccch_stirling.csv", "../data/stirling/charities_stirling.csv", "../data/stirling/maos_stirling.csv", "../data/stirling/dts_stirling.csv", (56.1165, -4.0), 9, "Stirling", "../data/stirling/businesses_stirling.csv", "../data/stirling/iz_info_stirling.csv", "../data/stirling/iz_info_dash_table_stirling.csv", "../assets/stirling_map.html")

In [52]:
# West Dunbartonshire
create_map("../data/west_dunbartonshire/ccch_west_dunbartonshire.csv", "../data/west_dunbartonshire/charities_west_dunbartonshire.csv", "../data/west_dunbartonshire/maos_west_dunbartonshire.csv", "../data/west_dunbartonshire/dts_west_dunbartonshire.csv", (55.9751, -4.5646), 11, "West Dunbartonshire", "../data/west_dunbartonshire/businesses_west_dunbartonshire.csv", "../data/west_dunbartonshire/iz_info_west_dunbartonshire.csv", "../data/west_dunbartonshire/iz_info_dash_table_west_dunbartonshire.csv", "../assets/west_dunbartonshire_map.html")

In [53]:
# West Lothian
create_map("../data/west_lothian/ccch_west_lothian.csv", "../data/west_lothian/charities_west_lothian.csv", "../data/west_lothian/maos_west_lothian.csv", "../data/west_lothian/dts_west_lothian.csv", (55.8818, -3.6267), 11, "West Lothian", "../data/west_lothian/businesses_west_lothian.csv", "../data/west_lothian/iz_info_west_lothian.csv", "../data/west_lothian/iz_info_dash_table_west_lothian.csv", "../assets/west_lothian_map.html")